In [1]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [13]:
IMAGE_SIZE = [299,299]

train_path = './archive/train/'
valid_path = './archive/test/'

In [14]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [5]:
for layer in inception.layers:
    layer.trainable = False

In [6]:
folders = glob('./archive/train/*')

In [7]:
folders

['./archive/train\\angry',
 './archive/train\\disgusted',
 './archive/train\\fearful',
 './archive/train\\happy',
 './archive/train\\neutral',
 './archive/train\\sad',
 './archive/train\\surprised']

In [15]:
x = Flatten()(inception.output)

In [16]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d_94 (Conv2D)          (None, 149, 149, 32)         864       ['input_2[0][0]']             
                                                                                                  
 batch_normalization_94 (Ba  (None, 149, 149, 32)         96        ['conv2d_94[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_94 (Activation)  (None, 149, 149, 32)         0         ['batch_normalization_94

In [21]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [22]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


In [23]:
training_set = train_datagen.flow_from_directory('./archive/train/',
                                                 target_size = (299, 299),
                                                 batch_size = 16,
                                                 class_mode = 'categorical')

Found 28709 images belonging to 7 classes.


In [24]:
test_set = test_datagen.flow_from_directory('./archive/test/',
                                            target_size = (299, 299),
                                            batch_size = 16,
                                            class_mode = 'categorical')

Found 7178 images belonging to 7 classes.


In [25]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\USER\AppData\Local\Temp\ipykernel_5020\177326590.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/10
1795/1795 [==============================] - 8075s 4s/step - loss: 2.0672 - accuracy: 0.2368 - val_loss: 3.3481 - val_accuracy: 0.2469
Epoch 2/10
 124/1795 [=>............................] - ETA: 1:51:22 - loss: 1.9525 - accuracy: 0.2389


KeyboardInterrupt



In [ ]:
import matplotlib.pyplot as plt

plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')